##import 

In [1]:
!git clone https://github.com/resemble-ai/Resemblyzer.git

fatal: destination path 'Resemblyzer' already exists and is not an empty directory.


In [2]:
!pip install resemblyzer

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
!cd Resemblyzer

In [4]:
# fpath = Path("100.091-1428999.wav")
# wav = preprocess_wav(fpath)

# encoder = VoiceEncoder()
# embed = encoder.embed_utterance(wav)
# np.set_printoptions(precision=3, suppress=True)
# print(embed)

In [4]:
!pip install-q pyannote.audio==1.1

ERROR: unknown command "install-q" - maybe you meant "install"


In [5]:
!pip install spectralcluster

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [6]:
import numpy as np

import pandas as pd
import timeit

import os
from scipy.signal.windows import hann
from datetime import datetime
import seaborn as sns
import matplotlib.pyplot as plt
sns.set_theme(style="darkgrid")

import torch
import torchaudio

from resemblyzer import preprocess_wav, VoiceEncoder
from pathlib import Path
import pickle
import scipy.io.wavfile
from spectralcluster import SpectralClusterer
from scipy.io.wavfile import read
from resemblyzer.audio import sampling_rate

## Model reresemblyzer

In [7]:
def walk_wav(directory:str):
    root, dirs, files = list(os.walk(directory))[0]
    # return files
    return [os.path.join(directory, file) for file in files if file.endswith('.wav') or file.endswith('.WAV') ]

In [ ]:
audio_list = walk_wav("/content/drive/MyDrive/iig.iiv /audio menu2")
audio_list = sorted(audio_list)
audio_list

In [8]:
audio_list_1 =("/content/17.086-1001102.wav")
audio_list_1

'/content/17.086-1001102.wav'

In [8]:
audio_list_test =walk_wav("/content/drive/MyDrive/iig.iiv /audio มี2 ไฟล์")
audio_list_test

['/content/drive/MyDrive/iig.iiv /audio มี2 ไฟล์/17.086-1001102.wav',
 '/content/drive/MyDrive/iig.iiv /audio มี2 ไฟล์/23.089-9476208.wav']

In [9]:
def create_labelling(labels, wav_splits):
    from resemblyzer.audio import sampling_rate
    sample_rate = sampling_rate
    print('sampling_rate:', sample_rate)
    times = [((s.start + s.stop) / 2) / sample_rate for s in wav_splits]
    labelling = []
    start_time = 0

    for i, time in enumerate(times):
        if i > 0 and labels[i] != labels[i - 1]:
            temp = ['speaker:'+str(labels[i - 1]), start_time, time]
            labelling.append(tuple(temp))
            start_time = time
        if i == len(times) - 1:
            temp = ['speaker:'+ str(labels[i]), start_time, time]
            labelling.append(tuple(temp))

    return labelling

In [10]:
# ชี้ path audio_list 
for i in audio_list_test:
  wav_fpath = Path(i)
  print(i)
  wav = preprocess_wav(wav_fpath)
  encoder = VoiceEncoder("cpu")
  _, cont_embeds, wav_splits = encoder.embed_utterance(wav, return_partials=True, rate = 8)
  print(cont_embeds.shape)   
  clusterer = SpectralClusterer(
      min_clusters=2,
      max_clusters=100,autotune=None,
      laplacian_type=None,
      refinement_options=None,
      custom_dist="cosine") #,
      # p_percentile=0.90),
      #gaussian_blur_sigma=1)

  labels = clusterer.predict(cont_embeds)

  speakers_labelled = create_labelling(labels, wav_splits)
   # print(speakers_labelled)
  speakers_labelle =pd.DataFrame(speakers_labelled)
   # speakers_report = i[len(path)+1:-4]
  ###speakers_labelle.to_csv(f'/content/drive/MyDrive/iig.iiv /1557 กด 2  model resember/{i.split("/")[-1]}.csv')
  speakers_labelle



/content/drive/MyDrive/iig.iiv /audio มี2 ไฟล์/17.086-1001102.wav
Loaded the voice encoder model on cpu in 0.02 seconds.
(1132, 256)
sampling_rate: 16000
/content/drive/MyDrive/iig.iiv /audio มี2 ไฟล์/23.089-9476208.wav
Loaded the voice encoder model on cpu in 0.01 seconds.
(864, 256)
sampling_rate: 16000


## Model  inaSpeechSegmenter

In [ ]:
!pip install inaSpeechSegmenter

In [20]:
from inaSpeechSegmenter import Segmenter
from inaSpeechSegmenter.export_funcs import seg2csv, seg2textgrid

In [ ]:
audio =("/content/17.086-1001102.wav")
# audio = sorted(audio)
audio

In [22]:
seg = Segmenter()
segmentation = seg(audio)
print(audio)
segmentation_labelle =pd.DataFrame(segmentation)
segmentation_labelle.columns = ['label','start','end']
print(segmentation)
len(segmentation)

/usr/local/lib/python3.7/dist-packages/pyannote/algorithms/utils/viterbi.py:88: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  for e, c in six.moves.zip(emission.T, consecutive)
/usr/local/lib/python3.7/dist-packages/pyannote/algorithms/utils/viterbi.py:97: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  for e, c in six.moves.zip(constraint.T, consecutive)


/content/17.086-1001102.wav
[('female', 0.0, 4.1), ('male', 4.1, 16.66), ('noEnergy', 16.66, 17.26), ('male', 17.26, 18.86), ('noEnergy', 18.86, 19.36), ('male', 19.36, 25.96), ('female', 25.96, 27.78), ('noEnergy', 27.78, 29.02), ('male', 29.02, 34.2), ('noEnergy', 34.2, 34.74), ('male', 34.74, 44.38), ('noEnergy', 44.38, 44.78), ('male', 44.78, 53.28), ('female', 53.28, 56.4), ('noEnergy', 56.4, 56.980000000000004), ('female', 56.980000000000004, 76.62), ('noEnergy', 76.62, 77.52), ('male', 77.52, 79.66), ('noEnergy', 79.66, 80.84), ('male', 80.84, 93.18), ('noEnergy', 93.18, 93.8), ('female', 93.8, 101.60000000000001), ('male', 101.60000000000001, 104.66), ('noEnergy', 104.66, 107.10000000000001), ('female', 107.10000000000001, 109.82000000000001), ('male', 109.82000000000001, 116.64), ('noEnergy', 116.64, 117.32000000000001), ('male', 117.32000000000001, 118.88), ('noEnergy', 118.88, 119.36), ('male', 119.36, 122.36), ('noEnergy', 122.36, 122.96000000000001), ('male', 122.960000000

46

In [ ]:
segmentation_labelle

In [ ]:
# #ไฟล์ audio_list menu2
# #speaker and genger identify
# #model inaSpeechSegmenter

# #========= 
# from inaSpeechSegmenter import Segmenter
# from inaSpeechSegmenter.export_funcs import seg2csv, seg2textgrid

# for i in audio_list :
#   seg = Segmenter()
#   segmentation = seg(i)
#   print(i)
#   segmentation_labelle =pd.DataFrame(segmentation)
#   segmentation_labelle.columns = ['label','start','end']
#   print(segmentation)
#   len(segmentation)
#   #speakers_report = i[len(path)+1:-4]
#   ##segmentation_labelle.to_csv(f'/content/drive/MyDrive/iig.iiv /1557 กด 2  model inaSpeechSegmenter/{i.split("/")[-1]}.csv')
#   # seg2textgrid(segmentation, 'myseg.TextGrid')


In [ ]:
# segmentation_labelle